In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
from sklearn import preprocessing

In [ ]:
df = pd.read_csv("../input/liberty-mutual-group-property-inspection-prediction/train.csv.zip")
#categorical cols
cols = df.columns
num_cols = df._get_numeric_data().columns
cate = list(set(cols) - set(num_cols) - set(["T1_V17","T2_V3","T2_V11","T2_V12"]))

#striping the values
cate_cols=[]
for X in cate:
    cate_cols.append(X.strip("''"))
cate_cols

temp_df =df

#encoding boolean cols
for X in ["T1_V17","T2_V3","T2_V11","T2_V12"]:
    le = preprocessing.LabelEncoder()
    temp_df[X] = le.fit_transform(temp_df[X])
temp_df


In [ ]:
fig = plt.figure(figsize=(20,10))
corr = temp_df.drop("Hazard",axis=1).corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values , annot=True)


In [ ]:
sns.barplot(y=temp_df["Hazard"] , x =temp_df["T1_V2"])

In [ ]:
fig = plt.figure(figsize=(80,60))
sns.barplot(y=temp_df["Hazard"] , x =temp_df["T2_V1"] )

In [ ]:
temp_df = temp_df.drop(["T1_V17"],axis=1)
temp_df

In [ ]:
#dummyfying the categorical cols
for X in cate_cols:
    temp_df= pd.concat([temp_df,pd.get_dummies(df[X],prefix=[X], drop_first = True  )], axis = 1)
    temp_df = temp_df.drop([X] , axis =1 )
temp_df

In [ ]:
fig = plt.figure(figsize=(80,60))
corr = temp_df.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values , annot=True)

In [ ]:
y = temp_df["Hazard"]
temp_df = temp_df.drop(["Id"],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(temp_df, y, test_size=0.25, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train ,y_train)
importance = lr.coef_
p_out = lr.predict(X_test)
from sklearn.metrics import mean_squared_error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, p_out,squared = False))

In [ ]:
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train ,y_train)
p_out = lr.predict(X_test)
from sklearn.metrics import mean_squared_error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, p_out,squared = False))

In [ ]:
ensemble_le = VotingRegressor(estimators = [('lin_r',lin_r_le),('xgb',xgb_le)])
ensemble_le.fit(X_train,y_train)
pred_ensemble_le = ensemble_le.predict(X_test)

In [ ]:
sub = pd.read_csv("../input/submission/Submission.csv")
sub.head()

In [ ]:
sub.to_csv("./SubFile.csv")

# **Analysis**



In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
from sklearn import preprocessing

In [ ]:
#Label encoding done on "T1_V17","T2_V3","T2_V11","T2_V12" and dropped T1_V17
df = pd.read_csv("../input/liberty-mutual-group-property-inspection-prediction/train.csv.zip")
#categorical cols
cols = df.columns
num_cols = df._get_numeric_data().columns
cate = list(set(cols) - set(num_cols) - set(["T1_V17","T2_V3","T2_V11","T2_V12"]))

#striping the values
cate_cols=[]
for X in cate:
    cate_cols.append(X.strip("''"))
cate_cols

temp_df =df

#encoding boolean cols
for X in ["T1_V17","T2_V3","T2_V11","T2_V12"]:
    le = preprocessing.LabelEncoder()
    temp_df[X] = le.fit_transform(temp_df[X])
temp_df = temp_df.drop(["T1_V17"] , axis =1)
temp_df

In [ ]:
num_cols = list(num_cols)
num_cols.remove('Id')
num_cols.remove('Hazard')

In [ ]:
sns.lineplot(data=temp_df , x ="T1_V1",y="Hazard")

In [ ]:
sns.lineplot(data=temp_df , x ="T1_V2",y="Hazard")


In [ ]:
fig = plt.figure(figsize=(20,10))
sns.lineplot(data=temp_df , x ="T1_V2",y="T1_V1")


In [ ]:
fig = plt.figure(figsize=(20,10))
sns.lineplot(data=temp_df , x ="T1_V1",y="Hazard",hue="T1_V4" , sort= True)

In [ ]:
#fig, axs = plt.subplots(ncols=2)
sns.lineplot(data=temp_df , x ="T1_V3",y="Hazard" )


In [ ]:
for X in cate_cols:
    print(temp_df[X].unique())

In [ ]:
for X in cate_cols:
    sns.catplot(data=temp_df , x = X , y ="Hazard")


# **hazard is <=25**
# 1. T1v11 --> M,A,F,N(mostly),D,K(mostly)
# 2. T1v4  --> H,E(mostly),C,S,G
# 3. T2v5  --> D,E,F(very less data points)
# 4. T2v13 --> B
# 5. t1v15 --> D,H,N,S,W,F
# 6. t1v16 --> D,C,F,H,A,Q,G,L,P
# 7. t1v5  --> D,J,  (E,L) have low density


In [ ]:
print((cate_cols))

In [ ]:
# lot of outliers
for X in cate_cols:
    sns.catplot(data=temp_df , x = X , y ="Hazard" , kind = "box")


In [ ]:
for X in num_cols:
    sns.catplot(data=temp_df , x = X , y="Hazard" , kind= "bar")

# **Hazard increase +ve or decreases -ve**
# 1. t1v1 +ve
# 2. t1v2 +ve
# 3. t1v3 , t2v14 ,t2v15 +ve
# 4. t1v14,t2v1 , t2v9-ve

In [ ]:
fig = plt.figure(figsize=(10,5))
for X in num_cols:
    sns.displot(temp_df[X] ,kde = True)
plt.show()

In [ ]:
temp_df.hist(figsize=(15,30),layout=(6,4))

# T1V5 to T1V9 Analysis

In [ ]:
import plotly.express as px

In [ ]:
fig = px.bar(df, x="T1_V1", y="Hazard", color="T1_V12", barmode = 'stack')
 
fig.show()#

In [ ]:
fig = px.bar(df, x="T1_V2", y="Hazard", color="T1_V5", barmode = 'stack')#why is hazard showing in thousands
 
fig.show()#

*When T1v12 is B then 90% of times Hazard =1*

In [ ]:
fig = px.bar(df, x="T1_V10", y="Hazard", color="T1_V5", barmode = 'stack')
 
fig.show()#

# Some Observations

T1v5 is L (5 values/59000) –haz(1,3,7,8)
1.	t1v7,t1v8,t1v12 are B
2.	t1v15,t2_v5 is A
3.	t1v17  , t2_v3 ,t2_v12 is N
4.	t1v9 D
5.	t2_v3 is E
6.	t2v8 1


t1v5 is E (5 values/59000)—haz(2,4,7,14)
1.	t1v7 ,t1v8 , t1v12 are B
2.	t1v15    is A
3.	t1v9 is D
4.	t1v17,t2v12 Is N
5.	t2v11 is Y
6.	t2v8 is 1


t1v5 is D (149)—haz(1-20)
1.	t1v7 is 98% ,t1v8 ,t1v12 is 80% B
2.	t1v9 is 80+% D
3.	t1v15 is 75% A
4.	t1v17 , t2v12 is 80% N
5.	t2v8 is 99% 1
6.	t2v11 is 80% Y


t1v5 is J (188)—haz(1-20)
1.	t1v7 is 98% ,t1v8 ,t1v12 is 80% B
2.	t1v9 is 87+%  E
3.	t1v15 is 90% , t2v5 100% A
4.	t1v17 is 98%  , t2v12 is 80% N
5.	t2v8 is 99% 1
6.	t2v11 is 77% Y


t1v5 is B (3335)—99% haz(1-25)
1.	t1v7 is 98% ,t1v8 ,t1v12 is 90% B
2.	t1v9 is 90% D
3.	t1v15 is 75% A
4.	t1v17 , t2v12 is 80% N
5.	t2v8 is 98% 1
6.	t2v11 is 66% Y


t1v5 is I (4297)—99% haz(1-25)
1.	t1v7 is 95% ,t1v8 ,t1v12 is 90% B
2.	t1v9 is 90% E
3.	t1v15 is 92% , t2v5 100%  A
4.	t1v17  is 76, t2v12 is 80% N
5.	t2v8 is 98% 1
6.	t2v11 is 78% Y


t1v5 is C (7845)—99% haz(1-25)
1.	t1v7 is 95% ,t1v8 ,t1v12 is 90% B
2.	t1v9 is 90% D
3.	t1v15 is 92% , t2v5 61%  A
4.	t1v17  is 80%, t2v12 is 80% N
5.	t2v8 is 98% 1
6.	t2v11 is 75% Y


t1v5 is H(10137)—99%-->haz(1-25)
1.	t1v7 is 94% ,t1v8 ,t1v12 is 90% B
2.	t1v9 is  D 39% and E is 48%
3.	t1v15 is 90% A
4.	t1v17  is 90%, t2v12 is 91% N
5.	t2v8 is 98% 1
6.	t2v11 is 57% Y


t1v5 is A(10900)—99%-->haz(1-25)
1.	t1v7 is 94% ,t1v8 ,t1v12 ,t1v11(85) is 90% B
2.	t1v9 is  D 86% and E is 48%
3.	t1v15 is 86 % , t2v5 68% A
4.	t1v17  is 80%, t2v12 is 80% N
5.	t2v8 is 98% 1
6.	t2v11 is 76% Y


t1v5 is K(14138)—99%-->haz(1-25)
1.	t1v7 is 94% ,t1v8 ,t1v12 is 90% B
2.	t1v9 is  E 90% 
3.	t1v15 is 90 % , t2v5 90% A
4.	t1v17  is 74%, t2v12 is 75% N
5.	t2v8 is 98% 1
6.	t2v11 is 76% Y


# Common Trend for Hazard (1-25)

* T1v7	T1v8	T1v12 - B
* T1v15 - A
* T1v17	T2v12 –  N
* T2v8 - 1
* T1v9 – D,E
* T2v11 - Y
